# Generation of Symbol Dataset

## Installation of required packages

In [ ]:
!pip install pillow
!pip install ipympl

## Script

### Import Internal Packages

In [ ]:
from os         \
    import      \
    walk,       \
    mkdir,      \
    listdir,    \
    remove

from os.path    \
    import      \
    join,       \
    isdir,      \
    isfile

from time \
    import time_ns

from random \
    import SystemRandom


### External Packages

In [ ]:
from PIL            \
    import          \
        ImageDraw,  \
        ImageFont

from PIL.Image  \
    import      \
        Image,  \
        new

### State Management

In [ ]:
is_debugging: bool = False

In [ ]:
output_path: str = '/tmp/output'
location_of_fonts: str = '/mnt/c/fonts'

In [ ]:
encoding: str = 'UTF-8'

In [ ]:
width: int = 512
height: int = 512

channels: str = 'RGB'

image = None

In [ ]:
batches: int = 1250
generate_letter: str = 'J'

In [ ]:
found_fonts: list = list()
size_of_found_fonts: int = 0
last_index_of_found_founds: int = 0

### Accessors

In [ ]:
def get_size_of_found_fonts() -> int:
    global size_of_found_fonts
    return size_of_found_fonts

def get_found_fonts() -> list:
    global found_fonts
    return found_fonts

def get_last_index_of_found_fonts() -> int:
    global last_index_of_found_founds
    return last_index_of_found_founds

In [ ]:
def is_debugging_now() -> bool:
    global is_debugging
    return is_debugging

def is_not_debugging() -> bool:
    return not is_debugging_now()

def get_encoding() -> str:
    global encoding
    return encoding

In [ ]:
def generate_filename() -> str:
    return str(time_ns()) + '.jpg'

In [ ]:
def select_random_value_in_color_range(
    generator: SystemRandom
) -> int:
    return generator.randint(
        0, 
        255
    )

def select_random_font() -> str:
    global found, last_index_of_found
    generator = SystemRandom()
    
    selection: int = generator.randint(
        0, 
        last_index_of_found
    )

    return found[selection]


def select_random_color() -> tuple:
    generator = SystemRandom()
    
    red = select_random_value_in_color_range(
        generator
    )
    
    green = select_random_value_in_color_range(
        generator
    )

    blue = select_random_value_in_color_range(
        generator
    )

    return (
        red, 
        green, 
        blue
    )


def select_random_font_size() -> int:
    global height
    generator = SystemRandom()
    
    return generator.randint(
        12, 
        height
    )

def select_random_quality() -> int:
    generator: SystemRandom = SystemRandom()
    return generator.randint(
        2, 
        85
    )

def start_of_index() -> int:
    return 0

def select_random_position_x() -> int:
    global width
    generator: SystemRandom =  SystemRandom()

    begin: int = start_of_index()
    end: int = width

    return generator.randint(
        begin, 
        end
    )

def select_random_position_y() -> int:
    global height
    generator: SystemRandom =  SystemRandom()

    begin: int = start_of_index()
    end: int = height

    return generator.randint(
        begin, 
        end
    )

def select_random_boolean() -> bool:
    generator: SystemRandom = SystemRandom()
    return bool(
        generator.getrandbits(
            1
        )
    )


### Generation & Procedures

In [ ]:
def generate_random_image(
    width: int, 
    height: int, 
    channel_mode: str
) -> Image:
    generator: SystemRandom = SystemRandom()
    
    image = new(
        mode=channel_mode, 
        size=(
            width, 
            height
        )
    )

    pixels = image.load()

    for x \
        in range(width):
        for y \
            in range(height):
            pixels[x, y] = (
                select_random_value_in_color_range(
                    generator
                ),
                select_random_value_in_color_range(
                    generator
                ),
                select_random_value_in_color_range(
                    generator
                )
            )

    return image

In [ ]:
def update_entire(
    image: Image
) -> None:
    global width, height
    pixels = image.load()

    for x in range(width):
        for y in range(height):
            pixels[x, y] = select_random_color()

In [ ]:
def update_by_samples(
    image: Image
) -> None:
    pixels = image.load()
    generator: SystemRandom = SystemRandom()
    
    last_index_in_row: int = width - 1
    last_index_in_column: int = height - 1

    selection_of_samples: int = int((width * height)/4)

    for i in range(selection_of_samples):
        selected_x: int = generator.randint(
            start_of_index(), 
            last_index_in_row
        )
        
        selected_y: int = generator.randint(
            start_of_index(), 
            last_index_in_column
        )

        pixels[selected_x, selected_y] = select_random_color()

In [ ]:
def update() -> None:
    global image
    
    if select_random_boolean():
        return update_by_samples(
            image
        )
    
    update_entire(
        image
    )

In [ ]:
def clear():
    global path
    if not isdir(path):
        mkdir(path)
    else:
        for file in listdir(path):
            full = join(
                path, 
                file
            )
            
            if isfile(full):
                remove(full)

In [ ]:
image = generate_random_image(
    width, 
    height, 
    channels
)

In [ ]:
if found is None or len(found) == start_of_index():
    for root, directories, files in walk(location_of_fonts):
        for file in files:
            full_path: str = join(root, file)
            found.append(full_path)

    size_of_found: int = len(found)
    last_index_of_found: int | None = (size_of_found - 1)

### Run the Process

In [ ]:
clear()

for i in range(batches):
    update()

    current_mirror = image.copy()
    context = ImageDraw.Draw(
        current_mirror
    )

    selected_font: str = select_random_font()
    selected_font_size: int = select_random_font_size()

    position_x = select_random_position_x()
    position_y = select_random_position_y()

    message_to_generate: str = generate_letter

    font_used = ImageFont.truetype(
        selected_font, 
        size=selected_font_size,
        encoding=get_encoding()
    )

    context.text(
        (position_x, position_y), 
        message_to_generate, 
        font=font_used, 
        fill=select_random_color(),
        align='center',
        anchor='ms',
        encoding=get_encoding(), 
        spacing=0.0
    )
    
    if is_debugging_now():
        display(current_mirror)
        break
    
    if is_not_debugging():
        full_path_to_save: str = join(
            path, 
            generate_filename()
        )

        current_mirror.save(
            full_path_to_save, 
            quality=select_random_quality()
        )

del current_mirror